In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
from crush_read import *
from crush_plot import *
plt.style.use('ggplot')
PATH

In [ ]:
study = study_outline(PATH)
targets = study_targets(PATH)
crushes = study_data(study)
crushes = modify(crushes)
crushes = calculate(crushes)

In [ ]:
_ = prep(crushes, targets)
crushes = refine(crushes, drop=False)

Prepare graphs needed for paper.

In [ ]:
# Convenience functions
def patient(patient):
    return crushes.loc[crushes['Patient'] == patient.upper(), :]

def tissue(tissue):
    return crushes.loc[crushes['Tissue'] == tissue.upper(), :]

def protocol(protocol):
    return crushes.loc[crushes['Protocol'] == protocol.upper(), :]

def load(load):
    return crushes.loc[crushes['Load'] == load.lower(), :]

In [ ]:
crushes

Focus on stop and first 10 patients.

In [ ]:
crushes = protocol('stop')
crushes = crushes[crushes['Patient'].isin(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])]

In [ ]:
crushes

In [ ]:
sb = tissue('sb')
sb.shape

In [ ]:
colon = tissue('colon')
colon.shape

In [ ]:
crushes['Patient'].unique()

# COLON PERCENTAGE PLOT

In [ ]:
damage = colon['Damage Score']
colon_dam = pd.DataFrame({'No Damage': (damage == 0),
                          'Minor Damage': (damage == 1),
                          'Major Damage': (damage == 2)})
colon_dam = pd.concat([colon_dam, colon['Load (g)']], axis=1)
colon_dam

In [ ]:
group = colon_dam.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group

In [ ]:
group[['Minor Damage', 'Major Damage']].plot.bar(width=0.8)

In [ ]:
group = group.reset_index()
group

In [ ]:
damage_types = ['No Damage', 'Minor Damage', 'Major Damage']
for damage in damage_types:
    group[damage] = group[damage] / group['Count']
group

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
colors = ListedColormap(['blue', 'orange', 'red'])
width = 50

for i, damage in enumerate(damage_types):
    plt.figure()
    plt.bar(x=group['Load (g)'], height=group[damage], width=width, label=damage, color=colors(i))
    plt.legend()
    plt.ylabel('Percentage of Samples')
    plt.xlabel('Load (g)')
    plt.ylim(0, 1)
    ax = plt.gca()
    ticks = ax.get_yticks()
    ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
colors = ListedColormap(['blue', 'orange', 'red'])
width = 50
shift = [-width, 0, width]
plt.figure()
for i, damage in enumerate(damage_types): 
    if i == 0:
        continue
    x_val = group['Load (g)'] + shift[i]
    plt.bar(x=x_val, height=group[damage], width=width, label=damage, color=colors(i))
plt.legend()
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
nice_colors = ListedColormap(['orangered', 'orange', 'dodgerblue'])

In [ ]:
plt.figure()
width = 100
types = list(reversed(damage_types))
for i, damage in enumerate(types):
    inputs = {'x': group['Load (g)'],
              'height': group[damage],
              'width': width,
              'label': damage + f' [{2 - i}]',
              'color': nice_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Trauma Score - Colon', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
# Same for serosal thickness change
delta = colon['Significant Serosal Change']
colon_del = pd.DataFrame({'No Change': (delta == False),
                          'Significant Change': (delta == True)})
colon_del = pd.concat([colon_del, colon['Load (g)']], axis=1)
group = colon_del.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()
delta_types = ['No Change', 'Significant Change']
for delta in delta_types:
    group[delta] = group[delta] / group['Count']
types = list(reversed(delta_types))

plt.figure()
for i, damage in enumerate(types):
    inputs = {'x': group['Load (g)'],
              'height': group[damage],
              'width': width,
              'label': damage,
              'color': nice_colors(i + 1)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Serosal Thickness Change - Colon', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

# SMALL BOWEL PERCENTAGE PLOT

In [ ]:
damage = sb['Damage Score']
sb_dam = pd.DataFrame({'No Damage': (damage == 0),
                       'Minor Damage': (damage == 1),
                       'Major Damage': (damage == 2)})
sb_dam = pd.concat([sb_dam, sb['Load (g)']], axis=1)
sb_dam

In [ ]:
group = sb_dam.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group

In [ ]:
group = group.reset_index()
group

In [ ]:
damage_types = ['No Damage', 'Minor Damage', 'Major Damage']
for damage in damage_types:
    group[damage] = group[damage] / group['Count']
group

In [ ]:
plt.figure()
types = list(reversed(damage_types))
for i, damage in enumerate(types):
    inputs = {'x': group['Load (g)'],
              'height': group[damage],
              'width': width,
              'label': damage + f' [{2 - i}]',
              'color': nice_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Trauma Score - Small Bowel', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
# Same for serosal thickness change
delta = sb['Significant Serosal Change']
sb_del = pd.DataFrame({'No Change': (delta == False),
                       'Significant Change': (delta == True)})
sb_del = pd.concat([sb_del, sb['Load (g)']], axis=1)
group = sb_del.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()
delta_types = ['No Change', 'Significant Change']
for delta in delta_types:
    group[delta] = group[delta] / group['Count']
types = list(reversed(delta_types))

plt.figure()
for i, damage in enumerate(types):
    inputs = {'x': group['Load (g)'],
              'height': group[damage],
              'width': width,
              'label': damage,
              'color': nice_colors(i + 1)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Serosal Thickness Change - Small Bowel', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
for i, c in group['Count'].items():
    print(group['Load (g)'][i])
    print(c)

# Patient Plots

In [ ]:
fig = plt.figure()
ax = plt.subplot2grid((20, 1), (1, 0), rowspan=19)